# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [1]:
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

import os

# Rutas de datasets y resultados
RUTA_DATASETS_BASE = "../datasets/datasets_aumentados/base/"
RUTA_DATASETS_AUMENTADOS = "../datasets/datasets_aumentados/"
RUTA_DATASETS_CLASICOS = "../datasets/datasets_aumentados/resampler_clasicos/"
DIRECTORIO_SALIDA = "../resultados"

os.makedirs(DIRECTORIO_SALIDA, exist_ok=True)
os.makedirs(RUTA_DATASETS_CLASICOS, exist_ok=True)


In [2]:
import gc, time  # gc: liberación explícita de memoria entre ejecuciones; time: medición de duración de búsquedas
from dataclasses import dataclass, asdict  # dataclass: estructura limpia para registrar resultados y metadatos de cada combinación

import numpy as np  # operaciones numéricas y manipulación de vectores/matrices
import pandas as pd  # manejo de estructuras tabulares (dataframes) para consolidar resultados


# Utilizamos validación estratificada + búsqueda aleatoria de hiperparámetros
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

# Métricas utilizadas en CV y test (todas macro para evitar sesgos por clase mayoritaria)
from sklearn.metrics import (
    f1_score,
    balanced_accuracy_score,
    recall_score,
    make_scorer
)

# Cada modelo se ejecuta dentro de un Pipeline para permitir transformaciones futuras
from sklearn.pipeline import Pipeline

# Modelo principal evaluado (Random Forest)
from sklearn.ensemble import RandomForestClassifier

# Suprimir warnings de convergencia innecesarios (SVM no se usa en esta fase)
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Controlar comportamiento en entornos con múltiples núcleos
# (evita paralelismo interno conflictivo con n_jobs de sklearn)
import os

# Estado aleatorio fijo para reproducibilidad entre ejecuciones
RANDOM_STATE = 42

# En Shuttle aumentado omitimos SVM por inestabilidad del ROC-AUC y límites computacionales
OMITIR_SVM_EN_SHUTTLE_AUMENTADO = True

# Archivo Excel consolidado con resultados CV y Test para todas las técnicas
NOMBRE_ARCHIVO_EXCEL = os.path.join(DIRECTORIO_SALIDA, "resultados_RS_cv_vs_test.xlsx")

In [3]:
from pathlib import Path
import re

# =========================
# Estructuras de datos
# =========================
@dataclass
class DatasetCombination:
    dataset_logico: str
    tipo_combination: str      # "base" | "clasico" | "pcsmote"
    ruta_train_csv: str
    ruta_test_csv: str
    tecnica_aumento: str = "base"
    valor_densidad: str = "--"
    valor_riesgo: str = "--"

    percentil_radio_distancia: str = "--",
    percentil_riesgo: str = "--"
    criterio_pureza: str = "--"
    umbral_densidad: str = "--"
    umbral_riesgo: str = "--"
    tipo_pureza: str = "--"

    grado_limpieza: str = "--"  # I0, I1, I5, etc.
    total_muestras_train: int | None = None
    tamanio_dataset: int | None = None  # tamaño total del dataset (train + test)
    sinteticos_generados: int = 0
    semillas_validas: int = 0
    
    tipo_pureza: str = "--"           # PE.. o Ppp.. del nombre de archivo
    nombre_configuracion: str = ""    # PRDxx_PRxx_CPxx_UDxxx_PE.._I.._SV.._SG..    

@dataclass
class RegistroRendimiento:
    dataset_logico: str
    tipo_combination: str
    nombre_modelo_aprendizaje: str
    tecnica_aumento: str
    valor_densidad: str
    valor_riesgo: str
    criterio_pureza: str
    grado_limpieza: str

    cantidad_train: int
    cantidad_test: int
    cantidad_caracteristicas: int

    # Métricas CV
    cv_f1_macro: float
    cv_balanced_accuracy: float
    cv_recall_macro: float    

    # Métricas Test
    test_f1_macro: float
    test_balanced_accuracy: float
    test_recall_macro: float  

    mejores_hiperparametros: str
    tiempo_busqueda_seg: float



def enumerar_combinaciones_base_y_aumentadas(
    ruta_base,
    ruta_clasicos,
    ruta_aumentados,
    verbose=True
):
    combinaciones = []
    cont_combinaciones = 0

    # Mapear (dataset_logico, grado_limpieza) → tamaño_train_base
    tamanio_train_base_por_dataset_y_I = {}

    # ==========================================================
    # 1) BASE
    #    train: {dataset}_I{I}_tm{n}_train.csv
    #    test : {dataset}_tm{n}_test.csv
    # ==========================================================
    if verbose:
        print(f"📂 Explorando carpeta base: {ruta_base}")

    archivos_base = os.listdir(ruta_base)

    for nombre in archivos_base:
        if not nombre.endswith("_train.csv"):
            if verbose:
                print(f"  ⚪ Omitido (no es *_train.csv): {nombre}")
            continue

        m = re.match(r"(.+?)_I(\d+)_tm(\d+)_train\.csv$", nombre)
        if not m:
            if verbose:
                print(f"  ⚪ No coincide patrón base con I*_tm*_train: {nombre}")
            continue

        dataset_logico = m.group(1)
        grado_limpieza = int(m.group(2))
        total_muestras_train = int(m.group(3))

        # Registrar tamaño de train base para este (dataset, I)
        clave_base = (dataset_logico, grado_limpieza)
        tamanio_train_base_por_dataset_y_I[clave_base] = total_muestras_train

        ruta_train_csv = os.path.join(ruta_base, nombre)

        # Buscar test correspondiente: {dataset}_tm{n}_test.csv
        patron_test = re.compile(rf"^{re.escape(dataset_logico)}_tdataset(\d+)_tm(\d+)_test\.csv$")
        nombre_test = None
        n_test_detectado = None

        for nombre_candidato in archivos_base:
            m_test = patron_test.match(nombre_candidato)
            if m_test:
                # Si hubiera más de uno, nos quedamos con el de mayor tm
                n_tm = int(m_test.group(2))
                tamanio_dataset=int(m_test.group(1))

                if n_test_detectado is None or n_tm > n_test_detectado:
                    n_test_detectado = n_tm
                    nombre_test = nombre_candidato

        if nombre_test is None:
            if verbose:
                print(f"  ⚠️  Falta test para dataset base '{dataset_logico}', se omite {nombre}")
            continue

        ruta_test_csv = os.path.join(ruta_base, nombre_test)

        cont_combinaciones += 1
        print(f"#{cont_combinaciones}  ✅ Agregado base: {nombre} combinado con {nombre_test}")

        combinaciones.append(DatasetCombination(
            dataset_logico=dataset_logico,
            tipo_combination="base",
            ruta_train_csv=ruta_train_csv,
            ruta_test_csv=ruta_test_csv,
            tecnica_aumento="base",
            valor_densidad=None,
            valor_riesgo=None,
            criterio_pureza=None,
            grado_limpieza=grado_limpieza,
            total_muestras_train=total_muestras_train,
            tamanio_dataset=tamanio_dataset
        ))

    # ==========================================================
    # 2) CLÁSICOS
    #    {tecnica}_{dataset}_I{I}_sg{sg}_train.csv
    #    test base: {dataset}_tm{n}_test.csv (mismo criterio que base)
    # ==========================================================
    if verbose:
        print(f"📂 Explorando carpeta clásicos: {ruta_clasicos}")

    archivos_clasicos = os.listdir(ruta_clasicos)

    for nombre in archivos_clasicos:
        if not nombre.endswith("_train.csv"):
            continue

        # ejemplo: adasyn_us_crime_I1_sg120_train.csv
        m = re.match(r"(.+?)_(.+?)_I(\d+)_sg(\d+)_train\.csv$", nombre)
        if not m:
            if verbose:
                print(f"  ⚠️  No cumple patrón clásicos: {nombre}")
            continue

        tecnica = m.group(1)
        dataset_logico = m.group(2)
        grado_limpieza = int(m.group(3))
        sinteticos_generados = int(m.group(4))

        # Recuperar tamaño de train base para este dataset y este I
        clave_base = (dataset_logico, grado_limpieza)
        total_muestras_train = tamanio_train_base_por_dataset_y_I.get(clave_base)

        if total_muestras_train is None:
            if verbose:
                print(
                    f"  ⚠️  No se encontró tamaño de train base para "
                    f"(dataset='{dataset_logico}', I={grado_limpieza}). Se omite {nombre}"
                )
            continue
        
        ruta_train_csv = os.path.join(ruta_clasicos, nombre)

        # Buscar test correspondiente en carpeta base
        patron_test = re.compile(rf"^{re.escape(dataset_logico)}_tdataset(\d+)_tm(\d+)_test\.csv$")
        nombre_test = None
        n_test_detectado = None

        for nombre_candidato in archivos_base:
            m_test = patron_test.match(nombre_candidato)
            if m_test:
                n_tm = int(m_test.group(2))
                tamanio_dataset = int(m_test.group(1))
                if n_test_detectado is None or n_tm > n_test_detectado:
                    n_test_detectado = n_tm
                    nombre_test = nombre_candidato

        if nombre_test is None:
            if verbose:
                print(f"  ⚠️  No hay test base para dataset '{dataset_logico}', se omite {nombre}")
            continue

        ruta_test_csv = os.path.join(ruta_base, nombre_test)

        cont_combinaciones += 1
        print(f"#{cont_combinaciones}  ✅ Agregado clásico: {nombre} combinado con {nombre_test}")

        combinaciones.append(DatasetCombination(
            dataset_logico=dataset_logico,
            tipo_combination="clasico",
            ruta_train_csv=ruta_train_csv,
            ruta_test_csv=ruta_test_csv,
            tecnica_aumento=tecnica.lower(),
            valor_densidad=None,
            valor_riesgo=None,
            criterio_pureza=None,
            grado_limpieza=grado_limpieza,
            total_muestras_train=total_muestras_train,
            sinteticos_generados=sinteticos_generados,
            tamanio_dataset=tamanio_dataset
        ))

    # ==========================================================
    # 3) PC-SMOTE (nuevo patrón)
    #
    # pcs_{dataset}_PRD{prd}_PR{pr}_CP{ent|prop}_UD{ud3}_{PE..|Ppp..}_I{iso}_SG{sg}_train.csv
    #
    # Ej:
    #   pcs_ecoli_PRD35_PR35_CPent_UD080_PE45_I0_SG120_train.csv
    #   pcs_ecoli_PRD35_PR35_CPprop_UD080_Ppp041_I0_SG007_train.csv
    #
    # valor_densidad  → percentil radio distancia (PRD)
    # valor_riesgo    → percentil riesgo (PR)
    # criterio_pureza → "entropia" / "proporcion"
    # grado_limpieza  → iso (I*)
    # sinteticos_generados → SG
    # ==========================================================
    if verbose:
        print(f"📂 Explorando carpeta aumentados: {ruta_aumentados}")

    archivos_aumentados = os.listdir(ruta_aumentados)

    patron_pcsmote = re.compile(
        r"^pcs_(?P<dataset>.+?)_"
        r"PRD(?P<prd>\d+)_"
        r"PR(?P<pr>\d+)_"
        r"CP(?P<cp>(?:ent|prop))_"
        r"UD(?P<ud>\d{3})_"
        r"(?P<tipo_pureza>(?:PE\d+|Upp\d{3}))_"
        r"UR(?P<ur>\d{3})_"
        r"I(?P<iso>\d+)_"
        r"SV(?P<sv>\d+)_"
        r"SG(?P<sg>\d+)_train\.csv$"
    )

    for nombre in archivos_aumentados:
        if not nombre.endswith("_train.csv"):
            continue

        m = patron_pcsmote.match(nombre)
        if not m:
            if verbose:
                print(f"  ⚪ Omitido (no es pcs válido): {nombre}")
            continue

        dataset_logico = m.group("dataset")
        valor_densidad = int(m.group("prd"))   # percentil radio distancia
        valor_riesgo   = int(m.group("pr"))    # percentil riesgo
        cp_code        = m.group("cp")         # "ent" | "prop"
        ud_str       = m.group("ud")        # umbral densidad en %, si después lo querés usar
        ur_str       = m.group("ur")        # umbral densidad en %, si después lo querés usar
        tipo_pureza = m.group("tipo_pureza")  # PE.. / Ppp.., si lo necesitás luego
        grado_limpieza = int(m.group("iso"))   # I*
        semillas_validas = int(m.group("sv"))
        sinteticos_generados = int(m.group("sg"))

        print(f"  ➡️  Descifrado pcsmote: dataset={dataset_logico}, prd={valor_densidad}, pr={valor_riesgo}, cp={cp_code}, ud={ud_str}, ur={ur_str}, tipo_pureza={tipo_pureza}, I={grado_limpieza}, sv={semillas_validas}, sg={sinteticos_generados}")

        if cp_code == "ent":
            criterio_pureza = "entropia"
        else:
            criterio_pureza = "proporcion"

        # nombre_configuracion EXACTO según el patrón
        nombre_configuracion = (
            f"PRD{valor_densidad}_"
            f"PR{valor_riesgo}_"
            f"CP{cp_code}_"
            f"UD{ud_str}_"
            f"UR{ur_str}_"
            f"{tipo_pureza}_"
            f"I{grado_limpieza}_"
            f"SV{semillas_validas}_"
            f"SG{sinteticos_generados}"
        )            

        ruta_train_csv = os.path.join(ruta_aumentados, nombre)

        # Buscar test correspondiente en carpeta base
        patron_test = re.compile(rf"^{re.escape(dataset_logico)}_tdataset(\d+)_tm(\d+)_test\.csv$")
        nombre_test = None
        n_test_detectado = None

        for nombre_candidato in archivos_base:
            m_test = patron_test.match(nombre_candidato)
            if m_test:
                n_tm = int(m_test.group(2))
                tamanio_dataset = int(m_test.group(1))
                if n_test_detectado is None or n_tm > n_test_detectado:
                    n_test_detectado = n_tm
                    nombre_test = nombre_candidato

        if nombre_test is None:
            if verbose:
                print(f"  ⚠️  No hay test base para dataset '{dataset_logico}', se omite {nombre}")
            continue

        ruta_test_csv = os.path.join(ruta_base, nombre_test)

        cont_combinaciones += 1
        print(f"#{cont_combinaciones}  ✅ Agregado pcsmote: {nombre} combinado con {nombre_test}")

        combinaciones.append(DatasetCombination(
            dataset_logico=dataset_logico,
            tipo_combination="pcsmote",
            ruta_train_csv=ruta_train_csv,
            ruta_test_csv=ruta_test_csv,
            tecnica_aumento="pcsmote",
            valor_densidad=valor_densidad,
            valor_riesgo=valor_riesgo,

            criterio_pureza=criterio_pureza,
            percentil_radio_distancia=valor_densidad,
            percentil_riesgo=valor_riesgo,  
            umbral_densidad=ud_str,
            umbral_riesgo=ur_str,

            grado_limpieza=grado_limpieza,
            sinteticos_generados=sinteticos_generados,
            semillas_validas=semillas_validas,
            tipo_pureza=tipo_pureza,                 
            nombre_configuracion=nombre_configuracion,    
            tamanio_dataset=tamanio_dataset
    
        ))

    if verbose:
        print(f"📊 Total combinaciones descubiertas: {len(combinaciones)}")

    return combinaciones


print("🔎 Enumerando combinaciones base y aumentadas...")

combinaciones = enumerar_combinaciones_base_y_aumentadas(
    ruta_base=RUTA_DATASETS_BASE,
    ruta_clasicos=RUTA_DATASETS_CLASICOS,
    ruta_aumentados=RUTA_DATASETS_AUMENTADOS,
    verbose=True
)

if not combinaciones:
    print("❌ No se encontraron combinaciones de datasets.")


datasets_con_base = {c.dataset_logico for c in combinaciones if c.tipo_combination == "base"}
if not datasets_con_base:
    print("❌ No hay datasets base para comparar.")



🔎 Enumerando combinaciones base y aumentadas...
📂 Explorando carpeta base: ../datasets/datasets_aumentados/base/
#1  ✅ Agregado base: telco_churn_I0_tm5634_train.csv combinado con telco_churn_tdataset7043_tm1409_test.csv
#2  ✅ Agregado base: telco_churn_I10_tm5070_train.csv combinado con telco_churn_tdataset7043_tm1409_test.csv
#3  ✅ Agregado base: telco_churn_I1_tm5577_train.csv combinado con telco_churn_tdataset7043_tm1409_test.csv
#4  ✅ Agregado base: telco_churn_I3_tm5464_train.csv combinado con telco_churn_tdataset7043_tm1409_test.csv
#5  ✅ Agregado base: telco_churn_I5_tm5352_train.csv combinado con telco_churn_tdataset7043_tm1409_test.csv
  ⚪ Omitido (no es *_train.csv): telco_churn_tdataset7043_tm1409_test.csv
📂 Explorando carpeta clásicos: ../datasets/datasets_aumentados/resampler_clasicos/
#6  ✅ Agregado clásico: adasyn_telco_churn_I0_sg2594_train.csv combinado con telco_churn_tdataset7043_tm1409_test.csv
#7  ✅ Agregado clásico: adasyn_telco_churn_I10_sg2411_train.csv combina

In [4]:
EXCLUIR_DATASETS = {
    "shuttle",
    "iris",
    "glass",
    "heart",
    "wdbc",
    "ecoli",
    "us_crime",
    "predict_faults",
    "gear_vibration",
    # "telco_churn",
}  # {"shuttle", "ecoli", ...} si quisieras excluir algo

def construir_lista_plana_de_tareas(model_registry,dataset_combinations, orden_modelos,
                                    excluir_datasets=EXCLUIR_DATASETS, verbose=True):
    """
    Crea una lista plana de tareas (modelo, combinación) y aplica políticas de exclusión.
    - excluir_datasets: conjunto de nombres de dataset (en minúsculas) a excluir por completo.
    - Mantiene la política existente de omitir SVM en Shuttle aumentado si está activa.
    """
    tareas = []
    excluidos_por_dataset = 0
    excluidos_por_politica_svm_shuttle = 0

    for nombre_modelo in orden_modelos:
        for combo in dataset_combinations:
            ds = combo.dataset_logico.lower()

            # 1) Excluir datasets completos (p. ej., shuttle)
            if ds in (excluir_datasets or set()):
                excluidos_por_dataset += 1
                continue

            # 2) Política original: omitir SVM en Shuttle aumentado
            if (OMITIR_SVM_EN_SHUTTLE_AUMENTADO and
                nombre_modelo == "SVM" and
                ds == "shuttle" and
                combo.tipo_combination == "aumentado"):
                excluidos_por_politica_svm_shuttle += 1
                continue

            tareas.append((nombre_modelo, combo))

    if verbose:
        print(f"🧮 Tareas planificadas: {len(tareas)} "
              f"(excluidos por dataset: {excluidos_por_dataset}, "
              f"por política SVM-Shuttle↑: {excluidos_por_politica_svm_shuttle})")
    return tareas

def construir_estimador_y_espacio_random_forest():
    est = Pipeline([
        ('classifier', RandomForestClassifier(
            random_state=RANDOM_STATE,
            n_jobs=1,
            bootstrap=True,
            oob_score=False,
            n_estimators=150,
            max_depth=None,
            max_features='sqrt',
            min_samples_split=2,
            min_samples_leaf=1,
            class_weight=None,
            criterion='gini'
        ))
    ])
    # De momento no uso espacio aleatorio para RF (space vacío)
    space = {
        "classifier__n_estimators": [150, 300],        # solo 2 valores
        "classifier__max_features": ["sqrt", "log2"],  # regla clásica vs alternativa
    }
    return est, space

REGISTRO_MODELOS = {
    "RandomForest": construir_estimador_y_espacio_random_forest,
}
ORDEN_MODELOS = ["RandomForest"]

tareas = construir_lista_plana_de_tareas(
    model_registry=REGISTRO_MODELOS,
    dataset_combinations=combinaciones,
    orden_modelos=ORDEN_MODELOS,
    excluir_datasets=EXCLUIR_DATASETS,
    verbose=True
)
total_tareas = len(tareas)
print(f"📦 Total de tareas planificadas: {total_tareas}")



🧮 Tareas planificadas: 172 (excluidos por dataset: 0, por política SVM-Shuttle↑: 0)
📦 Total de tareas planificadas: 172


In [5]:
# Scoring para RandomizedSearchCV
SCORING_REFIT = "f1_macro"
SCORING_MULTIPLE = {
    "f1_macro": "f1_macro",
    "balanced_accuracy": "balanced_accuracy",
    "recall_macro": make_scorer(recall_score, average="macro"),
}


def ejecutar_rs_y_comparar_cv_con_test(
    estimator,
    space,
    X_train,
    y_train,
    X_test,
    y_test,
    configuracion_busqueda,
    verbose=0,
):
    """
    Ejecuta RandomizedSearchCV y devuelve:
      - F1 macro (CV y Test)
      - Balanced Accuracy (CV y Test)
      - Recall macro (CV y Test)
    """
    inicio = time.perf_counter()

    search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=space,
        n_iter=configuracion_busqueda["n_iter"],
        scoring=SCORING_MULTIPLE,
        refit=SCORING_REFIT,
        cv=configuracion_busqueda["cv"],
        random_state=RANDOM_STATE,
        n_jobs=configuracion_busqueda["n_jobs"],
        verbose=verbose,
    )

    search.fit(X_train, y_train)
    elapsed = time.perf_counter() - inicio

    # =====================
    # Métricas CV (mejor candidato)
    # =====================
    cv_results = search.cv_results_
    best_idx = search.best_index_

    cv_f1       = float(cv_results["mean_test_f1_macro"][best_idx])
    cv_bacc     = float(cv_results["mean_test_balanced_accuracy"][best_idx])
    cv_recall_m = float(cv_results["mean_test_recall_macro"][best_idx])

    # =====================
    # Métricas en Test
    # =====================
    best_est = search.best_estimator_
    y_pred = best_est.predict(X_test)

    test_f1       = float(f1_score(y_test, y_pred, average="macro"))
    test_bacc     = float(balanced_accuracy_score(y_test, y_pred))
    test_recall_m = float(recall_score(y_test, y_pred, average="macro"))

    return dict(
        mejores_params=search.best_params_,
        tiempo=elapsed,
        cv=dict(
            f1=cv_f1,
            bacc=cv_bacc,
            recall_macro=cv_recall_m,
        ),
        test=dict(
            f1=test_f1,
            bacc=test_bacc,
            recall_macro=test_recall_m,
        ),
    )


In [ ]:
N_ITER_BUSQUEDA_POR_DEFECTO = 4

# =========================
# Utilidades de datos
# =========================
def cargar_matriz_caracteristicas_y_etiquetas_desde_csv(ruta_csv):
    """Lee un CSV y devuelve (X, y). Usa 'target' si existe, si no la última columna como y."""
    df = pd.read_csv(ruta_csv)
    if "target" in df.columns:
        X = df.drop(columns=["target"]).to_numpy(dtype=np.float32, copy=False)
        y = df["target"].to_numpy()
    else:
        X = df.iloc[:, :-1].to_numpy(dtype=np.float32, copy=False)
        y = df.iloc[:, -1].to_numpy()
    return X, y


def definir_configuracion_busqueda_para_dataset(X_train, nombre_dataset_logico, tipo_combination):
    """
    Define configuración de búsqueda:
      Shuttle aumentado -> CV=2
      Shuttle o n>=10000 -> CV=3
      resto -> CV=5
    """
    n_muestras = X_train.shape[0]
    es_shuttle = nombre_dataset_logico.lower() == "shuttle"

    if es_shuttle and tipo_combination != "base":
        cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    elif es_shuttle or n_muestras >= 10000:
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    else:
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    return {
        "cv": cv,
        "n_iter": N_ITER_BUSQUEDA_POR_DEFECTO,
        "n_jobs": 1,
    }

registros = []
inicio_total = time.perf_counter()

for idx, (nombre_modelo, combo) in enumerate(tareas, start=1):

    print(f"\n{'='*80}")
    print(f"🏁 [{idx}/{total_tareas}] Dataset: {combo.dataset_logico} | "
          f"Tipo: {combo.tipo_combination} | Modelo: {nombre_modelo}")
    print(f"📂 Train: {os.path.basename(combo.ruta_train_csv)}")

    # =====================
    # Cargar datos
    # =====================
    try:
        X_train, y_train = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_train_csv)
        X_test,  y_test  = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_test_csv)
    except Exception as e:
        print(f"❌ Error leyendo CSV: {e}")
        continue

    # =====================
    # Configuración búsqueda
    # =====================
    configuracion_busqueda = definir_configuracion_busqueda_para_dataset(
        X_train, combo.dataset_logico, combo.tipo_combination
    )

    print(f"⚙️  Configuración de búsqueda: "
          f"n_iter={configuracion_busqueda['n_iter']}, "
          f"folds={configuracion_busqueda['cv'].n_splits}, "
          f"n_jobs={configuracion_busqueda['n_jobs']}")

    # =====================
    # Estimador + espacio
    # =====================
    estimator, space = REGISTRO_MODELOS[nombre_modelo]()
    print("🚀 Iniciando RandomizedSearchCV...")

    # =====================
    # Ejecutar búsqueda
    # =====================
    try:
        resultados = ejecutar_rs_y_comparar_cv_con_test(
            estimator, space, X_train, y_train, X_test, y_test,
            configuracion_busqueda=configuracion_busqueda,
            verbose=1
        )
    except Exception as e:
        print(f"❌ Error durante la búsqueda: {e}")
        continue

    print(f"✅ Búsqueda completada en {resultados['tiempo']:.2f} s")
    print(f"📊 F1(CV): {resultados['cv']['f1']:.4f} | "
          f"F1(Test): {resultados['test']['f1']:.4f}")

    # =====================
    # Registrar resultados
    # =====================
    registros.append(asdict(RegistroRendimiento(
        dataset_logico=combo.dataset_logico,
        tipo_combination=combo.tipo_combination,
        nombre_modelo_aprendizaje=nombre_modelo,
        tecnica_aumento=combo.tecnica_aumento,
        valor_densidad=combo.valor_densidad,
        valor_riesgo=combo.valor_riesgo,
        criterio_pureza=combo.criterio_pureza,
        grado_limpieza=combo.grado_limpieza,

        cantidad_train=int(X_train.shape[0]),
        cantidad_test=int(X_test.shape[0]),
        cantidad_caracteristicas=int(X_train.shape[1]),

        # Métricas CV (3 decimales, robusto)
        cv_f1_macro=round(resultados["cv"]["f1"], 3) if resultados["cv"]["f1"] is not None else None,
        cv_balanced_accuracy=round(resultados["cv"]["bacc"], 3) if resultados["cv"]["bacc"] is not None else None,
        cv_recall_macro=round(resultados["cv"]["recall_macro"], 3) if resultados["cv"]["recall_macro"] is not None else None,

        # Métricas Test (3 decimales, robusto)
        test_f1_macro=round(resultados["test"]["f1"], 3) if resultados["test"]["f1"] is not None else None,
        test_balanced_accuracy=round(resultados["test"]["bacc"], 3) if resultados["test"]["bacc"] is not None else None,
        test_recall_macro=round(resultados["test"]["recall_macro"], 3) if resultados["test"]["recall_macro"] is not None else None,


        mejores_hiperparametros=str(resultados["mejores_params"]),
        tiempo_busqueda_seg=float(resultados["tiempo"]),
    )))

    gc.collect()



🏁 [1/172] Dataset: telco_churn | Tipo: base | Modelo: RandomForest
📂 Train: telco_churn_I0_tm5634_train.csv
⚙️  Configuración de búsqueda: n_iter=4, folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Búsqueda completada en 49.15 s
📊 F1(CV): 0.7112 | F1(Test): 0.7052

🏁 [2/172] Dataset: telco_churn | Tipo: base | Modelo: RandomForest
📂 Train: telco_churn_I10_tm5070_train.csv
⚙️  Configuración de búsqueda: n_iter=4, folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Búsqueda completada en 41.10 s
📊 F1(CV): 0.7390 | F1(Test): 0.7132

🏁 [3/172] Dataset: telco_churn | Tipo: base | Modelo: RandomForest
📂 Train: telco_churn_I1_tm5577_train.csv
⚙️  Configuración de búsqueda: n_iter=4, folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Búsqueda completada en 43.36 s
📊 F1(CV): 0.7154 | F1(Test): 0.7019

🏁 [4/

In [ ]:
# ----------------- DataFrame de resultados crudos -----------------
print("\n📊 Compilando resultados globales...")
df_resultados = pd.DataFrame(registros)

import re
from pathlib import Path

registros_tabla = []

# ============================
# Mapa: (dataset, grado_I) → train_base
# Ejemplo:  train_base_por_dataset_y_iso['ecoli'][0]  = 268   (I0)
#           train_base_por_dataset_y_iso['ecoli'][5]  = 254   (I5)
#           train_base_por_dataset_y_iso['ecoli'][10] = 241   (I10)
# ============================
train_base_por_dataset_y_iso = {}

for comb in combinaciones:
    if comb.tecnica_aumento == "base":
        ruta = comb.ruta_train_csv
        # tamaño train desde el nombre: ..._tmXXX_train.csv
        m_train = re.search(r"_tm(\d+)_train\.csv$", comb.ruta_train_csv)

        tamanio_dataset = comb.tamanio_dataset # tamaño total del dataset (train + test)

        if not m_train:
            print(f"[ERROR] No se pudo extraer train_size en BASE para ruta => {ruta}")
            print("        Patrón esperado: *_tmXXX_train.csv")            
            continue
        train_size = int(m_train.group(1))

        # grado de Isolation Forest: ..._I0_..., ..._I5_..., ..._I10_...
        m_iso = re.search(r"_I(\d+)_", comb.ruta_train_csv)
        if not m_iso:
            print(f"[WARN] No se pudo extraer grado_iso en BASE para ruta => {ruta}")
            print("       Se asigna grado_iso = 0 por default")
            grado_iso = 0
        else:
            grado_iso = int(m_iso.group(1))

        if comb.dataset_logico not in train_base_por_dataset_y_iso:
            print(f"[DEBUG] Inicializando diccionario para dataset_base = {comb.dataset_logico}")
            train_base_por_dataset_y_iso[comb.dataset_logico] = {}

        # -------------------------
        # Guardar resultado limpio
        # -------------------------
        train_base_por_dataset_y_iso[comb.dataset_logico][grado_iso] = train_size

        # -------------------------
        # DEBUG 4: confirmación
        # -------------------------
        print(
            f"[OK] BASE detectado: dataset={comb.dataset_logico}, "
            f"grado_iso={grado_iso}, train={train_size}, ruta={ruta}"
        )

print("\n[DEBUG] train_base_por_dataset_y_iso =", train_base_por_dataset_y_iso)

# ============================
# Construir tabla resumida
# ============================
for comb in combinaciones:
    tecnica = comb.tecnica_aumento           # base / smote / adasyn / borderlinesmote / pcsmote
    dataset = comb.dataset_logico

    # --- Tamaño test (común a todas las técnicas del dataset) ---
    m_test = re.search(r"_tm(\d+)_test\.csv$", comb.ruta_test_csv)
    test_size = int(m_test.group(1)) if m_test else None

    # --- Tamaño train (debe coincidir con el baseline del mismo grado I) ---
    train_size = None

    if tecnica == "base":
        # El baseline puede tomar directamente su propio tmXXX
        m_train = re.search(r"_tm(\d+)_train\.csv$", comb.ruta_train_csv)
        if m_train:
            train_size = int(m_train.group(1))

    else:
        # Para todas las técnicas aumentadas (smote / adasyn / borderline / pcsmote)
        # buscamos el grado I correspondiente y miramos el baseline.
        #   - clásicos: el grado se lee del nombre del csv de train
        #   - pcsmote: el grado se lee del nombre de configuración (PRD..._I0_... etc.)
        if tecnica == "pcsmote":
            fuente_iso = comb.nombre_configuracion
        else:  # smote / adasyn / borderlinesmote
            fuente_iso = comb.ruta_train_csv

        m_iso = re.search(r"_I(\d+)_", fuente_iso)
        grado_iso = int(m_iso.group(1)) if m_iso else 0

        train_size = train_base_por_dataset_y_iso.get(dataset, {}).get(grado_iso)

        if train_size is None:
            print(
                f"[DEBUG] Sin baseline para técnica={tecnica}, "
                f"dataset={dataset}, grado_I={grado_iso}. "
                f"train_size quedará como None. ruta_train={comb.ruta_train_csv}"
            )

    # Total = train + test (solo si ambos están definidos)
    if train_size is not None and test_size is not None:
        total = train_size + test_size
    else:
        total = None

    # --- Configuración y síntesis ---
    if tecnica == "pcsmote":
        configuracion = comb.nombre_configuracion
        semillas_candidatas = comb.semillas_validas
        sinteticos_generados = comb.sinteticos_generados
    elif tecnica in ("smote", "adasyn", "borderlinesmote"):
        configuracion = Path(comb.ruta_train_csv).stem
        semillas_candidatas = None
        sinteticos_generados = comb.sinteticos_generados
    else:  # base
        configuracion = Path(comb.ruta_train_csv).stem
        semillas_candidatas = "-------"
        sinteticos_generados = "-------"

    # --- Métricas CV/Test para ESTA combinación concreta ---
    if comb.tipo_combination == "pcsmote":
        df_fila = df_resultados[
            (df_resultados["dataset_logico"]   == comb.dataset_logico) &
            (df_resultados["tipo_combination"] == comb.tipo_combination) &
            (df_resultados["grado_limpieza"]   == comb.grado_limpieza) &
            (df_resultados["valor_densidad"]   == comb.valor_densidad) &
            (df_resultados["valor_riesgo"]     == comb.valor_riesgo) &
            (df_resultados["criterio_pureza"]  == comb.criterio_pureza)
        ]
    else:
        # baseline / smote / adasyn / borderline
        df_fila = df_resultados[
            (df_resultados["dataset_logico"]   == comb.dataset_logico) &
            (df_resultados["tipo_combination"] == comb.tipo_combination) &
            (df_resultados["grado_limpieza"]   == comb.grado_limpieza)
        ]

    # --- Extraer métricas ---
    if df_fila.empty:
        f1_cv   = None
        ba_cv   = None
        f1_test = None
        ba_test = None
    else:
        fila = df_fila.iloc[0]
        f1_cv   = fila["cv_f1_macro"]
        ba_cv   = fila["cv_balanced_accuracy"]
        f1_test = fila["test_f1_macro"]
        ba_test = fila["test_balanced_accuracy"]


    registros_tabla.append({
        "Tecnica": tecnica,
        "Database": dataset,
        # "Configuracion": configuracion,

        "percentil radio distancia": comb.percentil_radio_distancia if tecnica == "pcsmote" else "--",
        "percentil riesgo": comb.percentil_riesgo if tecnica == "pcsmote" else "--",
        "Criterio pureza": comb.criterio_pureza if tecnica == "pcsmote" else "--",
        "umbral densidad": comb.umbral_densidad if tecnica == "pcsmote" else "--",
        "umbral riesgo": comb.umbral_riesgo if tecnica == "pcsmote" else "--",
        "Tipo pureza": comb.tipo_pureza if tecnica == "pcsmote" else "--",

        "Total N": comb.tamanio_dataset,
        "Grado Isolation Forest": comb.grado_limpieza,
        "Total Isolation": total,
        "Test": test_size,
        "Train": train_size,
        "Semillas candidatas train": semillas_candidatas,
        "Sinteticas generadas": sinteticos_generados,
        "F1 macro CV": f1_cv,
        "Balanced Accuracy CV": ba_cv,
        "F1 macro Test": f1_test,
        "Balanced Accuracy Test": ba_test,
    })

df_tabla_final = pd.DataFrame(registros_tabla)

print("\n[DEBUG] Vista rápida de df_tabla_final (solo heart y glass):")
print(
    df_tabla_final[
        df_tabla_final["Database"].isin(["glass", "heart"])
    ][["Tecnica", "Database", "Train"]]
)

df_tabla_final.to_excel(NOMBRE_ARCHIVO_EXCEL, index=False)

fin_total = time.perf_counter()
duracion = round(fin_total - inicio_total, 2)
print(f"\n🏁 Ejecución total completada en {duracion} s")
print(f"📘 Archivo Excel generado: {NOMBRE_ARCHIVO_EXCEL}")
